In [ ]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import os
import csv


save_dir = os.path.join(os.getcwd(), 'saved_models')

def str_to_bool(s):
    if s == 'True':
         return True
    elif s == 'False':
         return False
    else:
         raise ValueError

with open('config.csv', 'rt') as csvfile:
	configreader = csv.reader(csvfile, delimiter=',', quotechar='|')
	next(configreader)
	for row in configreader:
		print(row)
		[batch_size,num_classes,epochs,data_augmentation,num_predictions,model_name,steps_per_epoch,featurewise_center,samplewise_center,featurewise_std_normalization,samplewise_std_normalization,zca_whitening,rotation_range,width_shift_range,height_shift_range,horizontal_flip,vertical_flip] = row
		batch_size = int(batch_size) 
		num_classes = int(num_classes) 
		epochs = int(epochs) 
		data_augmentation = str_to_bool(data_augmentation) 
		num_predictions = int(num_predictions) 
		steps_per_epoch = int(steps_per_epoch)
		featurewise_center = str_to_bool(featurewise_center)
		samplewise_center = str_to_bool(samplewise_center)
		featurewise_std_normalization = str_to_bool(featurewise_std_normalization)
		samplewise_std_normalization = str_to_bool(samplewise_std_normalization)
		zca_whitening = str_to_bool(zca_whitening)
		rotation_range = int(rotation_range)
		width_shift_range = float(width_shift_range)
		height_shift_range = float(height_shift_range)
		horizontal_flip = str_to_bool(horizontal_flip)
		vertical_flip = str_to_bool(vertical_flip)



		# The data, shuffled and split between train and test sets:
		(x_train, y_train), (x_test, y_test) = cifar10.load_data()
		print('x_train shape:', x_train.shape)
		print(x_train.shape[0], 'train samples')
		print(x_test.shape[0], 'test samples')

		# Convert class vectors to binary class matrices.
		y_train = keras.utils.to_categorical(y_train, num_classes)
		y_test = keras.utils.to_categorical(y_test, num_classes)

		model = Sequential()
		model.add(Conv2D(32, (3, 3), padding='same',
						 input_shape=x_train.shape[1:]))
		model.add(Activation('relu'))
		model.add(Conv2D(32, (3, 3)))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		model.add(Conv2D(64, (3, 3), padding='same'))
		model.add(Activation('relu'))
		model.add(Conv2D(64, (3, 3)))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation('relu'))
		model.add(Dropout(0.5))
		model.add(Dense(num_classes))
		model.add(Activation('softmax'))

		# initiate RMSprop optimizer
		opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

		# Let's train the model using RMSprop
		model.compile(loss='categorical_crossentropy',
					  optimizer=opt,
					  metrics=['accuracy'])

		x_train = x_train.astype('float32')
		x_test = x_test.astype('float32')
		x_train /= 255
		x_test /= 255

		if not data_augmentation:
			print('Not using data augmentation.')
			model.fit(x_train, y_train,
					  batch_size=batch_size,
					  steps_per_epoch = steps_per_epoch,
					  epochs=epochs,
					  validation_data=(x_test, y_test),
					  shuffle=True)
		else:
			print('Using real-time data augmentation.')
			# This will do preprocessing and realtime data augmentation:
			datagen = ImageDataGenerator(
				featurewise_center=featurewise_center,  # set input mean to 0 over the dataset
				samplewise_center=samplewise_center,  # set each sample mean to 0
				featurewise_std_normalization=featurewise_std_normalization,  # divide inputs by std of the dataset
				samplewise_std_normalization=samplewise_std_normalization,  # divide each input by its std
				zca_whitening=zca_whitening,  # apply ZCA whitening
				rotation_range=rotation_range,  # randomly rotate images in the range (degrees, 0 to 180)
				width_shift_range=width_shift_range,  # randomly shift images horizontally (fraction of total width)
				height_shift_range=height_shift_range,  # randomly shift images vertically (fraction of total height)
				horizontal_flip=horizontal_flip,  # randomly flip images
				vertical_flip=vertical_flip)  # randomly flip images
			# Configure batch size and retrieve one batch of images
			img_rows = 32
			img_cols = 32
			for X_batch, y_batch in datagen.flow(x_train, y_train, batch_size=9):
			# Show 9 images
				for i in range(0, 9):
					plt.subplot(330 + 1 + i)
					plt.imshow(X_batch[i].reshape(img_rows, img_cols, 3))
				# show the plot
				plt.show()
				break

			# Compute quantities required for feature-wise normalization
			# (std, mean, and principal components if ZCA whitening is applied).
			datagen.fit(x_train)

			# Fit the model on the batches generated by datagen.flow().
			history = model.fit_generator(datagen.flow(x_train, y_train,
											 batch_size=batch_size),
								steps_per_epoch = steps_per_epoch,
								epochs=epochs,
								validation_data=(x_test, y_test),
								workers=4)
			
		# list all data in history
		print(history.history.keys())
		# summarize history for accuracy
		plt.plot(history.history['acc'])
		plt.plot(history.history['val_acc'])
		plt.title('model accuracy')
		plt.ylabel('accuracy')
		plt.xlabel('epoch')
		plt.legend(['train', 'test'], loc='upper left')
		plt.show()
		# summarize history for loss
		plt.plot(history.history['loss'])
		plt.plot(history.history['val_loss'])
		plt.title('model loss')
		plt.ylabel('loss')
		plt.xlabel('epoch')
		plt.legend(['train', 'test'], loc='upper left')
		plt.show()

		# Save model and weights
		if not os.path.isdir(save_dir):
			os.makedirs(save_dir)
		model_path = os.path.join(save_dir, model_name)
		model.save(model_path)
		print('Saved trained model at %s ' % model_path)

		# Score trained model.
		scores = model.evaluate(x_test, y_test, verbose=1)
		print('Test loss:', scores[0])
		print('Test accuracy:', scores[1])